# Chapter 13. Interfaces, Protocols, and ABCs

"Program to an interface, not an implementation."

OOP is all about interfaces.

Ở Python 3.8, có 4 cách để định nghĩa interfaces, được mô tả trong Typing Map

- Duck typing (Chap 1)
    - Default
    - Tư tưởng: "If it walks like a duck and quacks like a duck, it must be a duck".
        -  Kiểu hoặc lớp (class) của một đối tượng (object) ít quan trọng hơn phương thức mà nó định nghĩa.
- Goose typing (Chap 13)
    - Supported by abstract base classes (ABCs)
    - Dựa trên runtime check ABCs của object
- Static typing (chap 8, 15)
    - Hướng tiếp cận của statical languages (Java, C)
    - Hỗ trợ bởi `typing` module
    - Enforced by external type checkers
- Static duck typing
    - Phổ biến ở Go
    - Hỗ trợ bởi `typing.Protocol`
    - Enforced by external type checkers

![image.png](image/13-1.png)

Chương này nói về duck typing, goose typing, and static duck typing
- static typing chỉ sử dụng các types cụ thể thay vì interface abstractions

## Two Kinds of Protocols

So sánh 2 loại Typing (bên trái) với Protocol

- Protocols
    - VD: Trong network protocols là `GET`, `PUT`
    - VD: Trong sequence protocol - method cho phép object như 1 seq
        - `__getitem__` method
        - `__len__` method (optional)
    - --> Protocol is an “informal interface.”

- Dynamic protocol
    - informal protocols
    - implicit (ngầm định)
- Static protocal
    - từ Python 3.8 (static duck tying)
    - explicit (rõ ràng): `typing.Protocol` subclass, ABCs

2 điểm khác biệt chính giữa 2 loại:
- Implement
    - Dynamic protocal: Object có thể chỉ cần impl 1 phần mà vẫn useful
    - Static protocol: Object phải impl mọi method trong protocal class
- Verify
    - Dynamic protocal: NaN
    - Static protocol: By static type checkers

The rest of this chapter covers dynamic and static protocols, as well as ABCs.

## Programming Ducks

Deep driver
Make it safer while preserving its major strength (flexibility)

Bắt đầu dynamic protocols với __sequence__ và __iterable__ protocols.

### Python Digs(đào) Sequences

Triết lý của Python Data Model là work với nhiều dynamic protocols thiết yếu nhất có thể.

Hình 13-2 cho thấy cách `Sequence` interface được chính thức hóa dưới dạng ABC. Mô tả những gì mà một full-fledged `Sequence` sẽ hỗ trợ.

![image.png](image/13-2.png)

Từ 13-2, thấy rằng subclass đúng của Seg phải imple `__getitem__` và `__len__`. Những methods khác là cụ thể -> subclass sẽ kế thừa nó/viết lại 1 cái phù hợp hơn.
- VD: Khi ko có `__iter__`, Python dự phòng rằng nếu có `__getitem__`, nó sẽ chạy index start ở 0.

TIP

The ABCs are useful as starting points for new classes, and to support explicit type checking at runtime (a.k.a. goose typing) as well as type hints for static type checkers.

Ví dụ sau không là subclass abc.Sequence, nhưng imple `__getitem__` và `__len__`

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

### Monkey Patching: Implementing a Protocol at Runtime

Monkey patching is dynamically changing a module, class, or function at runtime, to add features or fix bugs.

`FrenchDeck` ở trên thiếu đi tính năng `shuffle`. Nhưng theo Pythonic, vì ta đã làm nó act như 1 Seq -> nó ko cần viết 1 hàm shuffle của nó, mà có thể dùng `random.shuffle`

TIP

Khi bạn tuân theo các protocols đã được thiết lập, bạn sẽ cải thiện cơ hội tận dụng thư viện tiêu chuẩn và mã của bên thứ ba,, thanks to duck typing.

In [5]:
# shuffle operates in place, by swapping items inside the collection
from random import shuffle
deck = FrenchDeck()
list(deck)

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='J', suit='spades'),
 Card(rank='Q', suit='spades'),
 Card(rank='K', suit='spades'),
 Card(rank='A', suit='spades'),
 Card(rank='2', suit='diamonds'),
 Card(rank='3', suit='diamonds'),
 Card(rank='4', suit='diamonds'),
 Card(rank='5', suit='diamonds'),
 Card(rank='6', suit='diamonds'),
 Card(rank='7', suit='diamonds'),
 Card(rank='8', suit='diamonds'),
 Card(rank='9', suit='diamonds'),
 Card(rank='10', suit='diamonds'),
 Card(rank='J', suit='diamonds'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='K', suit='diamonds'),
 Card(rank='A', suit='diamonds'),
 Card(rank='2', suit='clubs'),
 Card(rank='3', suit='clubs'),
 Card(rank='4', suit='clubs'),
 Card(rank='5', suit='clubs'),
 Card(rank='6', 

In [6]:
shuffle(deck)
# error vì FrenchDeck chỉ imple immutable seq protocal --> không thể shuffle in place
# --> Muốn nó là mutable seq, cần cung cấp __setitem__

TypeError: 'FrenchDeck' object does not support item assignment

In [8]:
# Fix nó ở runtime, tương tác với console
# deck, position, card thay vì self, key, value để chỉ rằng methods nên bắt đầu từ 1 func đơn giản và đặt tên tham số đầu là self chỉ là 1 quy ước --> đặt vậy OK vì mình viết nó ở console.
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card # <-- Monkey Patching
shuffle(deck)
deck[:5]

[Card(rank='K', suit='spades'),
 Card(rank='J', suit='diamonds'),
 Card(rank='Q', suit='clubs'),
 Card(rank='9', suit='hearts'),
 Card(rank='9', suit='diamonds')]

Phần tiếp theo show một số code patterns để phát hiện các dynamic protocols mà không cần dùng đến các kiểm tra rõ ràng.

### Defensive Programming and “Fail Fast”

In a dynamically typed language, “fail fast” is excellent advice for safer and easier-to-maintain programs.
- Failing fast means raising runtime errors as soon as possible
    - VD: rejecting invalid arguments right a the beginning of a function body.



In [ ]:
# VD
# Ngay lập tức build list từ iterable
# Nếu iterable sai kiểu , ngay lập tức fail fast: TypeError
def __init__(self, iterable):
    self._balls = list(iterable)

# Nếu không làm vậy, chương trình sẽ vẫn raise khi 1 hàm nào đó gọi self._balls mà không phải là list --> khi đó khó tìm ra root cause.

# Tuy nhiên, gọi list() có thể tệ nếu ta cần change nó in-place như ví dụ shuffle.
# Khi đó, runtime check `isinstance(x, abc.MutableSequence)` hoặc `len()` sẽ cần đến

Ở ví dụ trên, type hint sẽ catch đc problem sớm hơn, nhưng không phải catch đc mọi thứ.
- `Any` type
- Không được enforced ở runtime


--> Fail fast là tuyến phòng thủ cuối

In [ ]:
# Ví dụ 2: Defensive code + duck typing để handle việc thay đổi biến field_names

# Example 13-5. Duck typing to handle a string or an iterable of strings
field_names = 'huy, mac'
try:
        field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)
if not all(s.isidentifier() for s in field_names):
    raise ValueError('field_names must all be valid identifiers')

--> duck typing is more expressive than static type hints
```python
def namedtuple(
        typename: str,
        field_names: Union[str, Iterable[str]],
        *,
        # rest of signature omitted
```

Giờ, move sang loại runtime type check rõ ràng hơn: Goose typing

## Goose Typing

ABCs for runtime type check

Python doesn’t have an `interface` keyword. We use `abstract base classes (ABCs`) to define interfaces for explicit type checking at runtime—also supported by static type checkers.
- ABCs introduce virtual subclasses
    - which are classes that don’t inherit from a class but are still recognized by `isinstance()` and `issubclass()`

### Subclassing an ABC


## Static Protocols

Covers usage, implementation, and design of `typing.Protocol` subclasses